In [45]:

import numpy as np
import scipy.io as scio
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import os
import skimage
import sklearn.model_selection
from sklearn.linear_model import LogisticRegression
import seaborn as sns
sns.set()
from sklearn import metrics

In [41]:
HR_Distance = np.genfromtxt("HR_Distances.csv", delimiter=",")
EDA_Distance = np.genfromtxt("EDA_Distances.csv", delimiter=",")


HR_Distance_Windows_extract = np.genfromtxt("HR_Distances_Windows.csv", delimiter=",")
EDA_Distance_Windows_extract = np.genfromtxt("EDA_Distances_Windows.csv", delimiter=",")
HR_Distance_Windows = np.empty((15,73,73))
EDA_Distance_Windows = np.empty((15,73,73))
ms_interval_labels = np.genfromtxt("MS_Interval_Labels.csv", delimiter=",")
ms_end_labels = np.genfromtxt("MS_EndLabels.csv", delimiter=",")






for i in range(15):
    HR_Distance_Windows[i,:,:] = HR_Distance_Windows_extract[:,73*i:73*(i+1)]
    EDA_Distance_Windows[i,:,:] = EDA_Distance_Windows_extract[:,73*i:73*(i+1)]
    
    

In [65]:
ms_interval_labels.shape

(15, 73)

In [34]:
def distance_laplace(distance_mat, scale_up=1, smooth_out=1):
    # Attempt of a function w/ hyperparameters scale up and smooth out, elementwise scale_up / (x^2 + smooth_down), t
    #take reciprocal of distances to convert low distances -> higher numbers and vice versa to try and encourage learning the
    # smallest distance values. 
    # Inspired by CID metric
    
    if smooth_out < 0:
        raise ValueError("Smooth Out Can't Be Negative")
        
    if scale_up < 0:
        raise ValueError("Smooth Out Can't Be Negative")
    
    dist_mat =  scale_up * np.exp(-1 * distance_mat / smooth_out) # Laplace Distribution 
    np.fill_diagonal(dist_mat, 0) # Want diagonal to equal 0 so we try not to learn the diagonal
    
    
    
    return dist_mat

def LoCo(Ux, Uy):
    USx, _, _ = np.linalg.svd(Ux, full_matrices=True)
    USy, _, _ = np.linalg.svd(Uy, full_matrice = True)
    return .5 * np.linalg.norm(Ux.T @ USy[:,1], ord = 2) + .5 * np.linalg.norm(Uy.T @ USx[:,1], ord = 2)

def FOTS(Ux, Uy):
    return np.linalg.norm(Ux - Uy, ord=frob)

# Soft Threshold function
def soft(z, lam):     
    return np.sign(z)*np.maximum(np.abs(z)-lam,0) 

# L21 proximal function (HW) 
def l21_prox(z, lam):
    """compute the proximal operator of L_21 norm with argument z and paramter lam"""
    x = z.copy()
    col_norm = np.linalg.norm(z,axis=0)
    idx = col_norm<=lam 
    idx_c = np.logical_not(idx)
    x[:,idx]=0
    x[:,idx_c]=(1-lam/col_norm[idx_c])*x[:,idx_c]
    return x
#ADMM Implementation of l2,1 norm-Robust PCA for summed datasets
def trioADMM_L21(L, SE, SH, E, H, lam, rho, niter=10): 
    '''
    Input: 
        L:     Shared Low Rank Component of the HR and EDA Sum Data Matrix 
        SE:    The Sparse Component of the EDA Data Matrix
        SH:    The Sparse Component of the HR Data Matrix
        E:     The EDA Data Matrix
        H:     The HR Data Matrix
        lam:   The regularization term 
        rho:   Augmented Lagrangian Parameter 
        niter: Number of Iterations 

    Intermediate: 
        W:     The scaled Dual variable 
        Z:     The summed matrix

    Output: 
        L:     The Shared Low Rank Component of the Data Matrix 
        SE:     The sparse component of the EDA data matrix (or can be thought of as EDA background noise)
        SH:     The sparse component of the HR data matrix (or can be thought of as HR background noise)
    '''
    Z = E + H
    W = Z-L-SE - SH
    for itr in range(niter):
        U,Sig,V = np.linalg.svd(Z-SE - SH +W, full_matrices=False)
        L = .5 * np.dot(np.dot(U,np.diag(soft(Sig,1/rho))),V)
        SE = l21_prox(Z - L - SH - W, lam/rho)
        SH = l21_prox(Z - L - SE - W, lam/rho)
        W = Z-L-SE - SH+W
    return L,SE, SH

In [47]:
def triplePCA(Rho, k, HR_Data, EDA_Data,ms_labels,ld = 1, printfigs = True):
    #Rho = 1
    #k = 67
    
    Z_init = (HR_Data + EDA_Data).T
    x_hat, noise_E, noise_H = trioADMM_L21(L=Z_init, SE = np.zeros(Z_init.shape), SH = np.zeros(Z_init.shape), E = EDA_Data.T, H = HR_Data.T, lam=ld, rho=Rho)

    #shared s_hat
    x_rank = np.linalg.matrix_rank(x_hat)
    if printfigs:
        print("Rank of original data matrix is: {}".format(np.linalg.matrix_rank(Z_init)))
        print("Rank of low rank data matrix is: {}".format(x_rank))
   
   
    if x_rank < k:
        print('k set too high for out matrix of rank: ' + str(x_rank))
    k = min(x_rank,k)
    U,S,V = np.linalg.svd(x_hat.T, full_matrices=False)
    x_rpca = np.dot(x_hat,U[:,:k]).T

    num_train = int(num_subjects * train_split)
    num_test = num_subjects - num_train
    #x_tilde = np.concatenate([np.ones([1, num_subjects]), x_rpca], axis=0)

    sum_train, sum_test, ms_sum_train, ms_sum_test = sklearn.model_selection.train_test_split(x_rpca.T, ms_labels, test_size = num_test, random_state=2024)
    logRegr = LogisticRegression(penalty ='l1',solver='liblinear', max_iter=1000, warm_start=True)
    logRegr.fit(sum_train, ms_sum_train)
    score = logRegr.score(sum_test, ms_sum_test)
    #print(score)
    predicted = logRegr.predict(sum_test)
    rand = sklearn.metrics.adjusted_rand_score(ms_sum_test, predicted)
    #print(rand)
    if printfigs:
        fig, axes = plt.subplots(1,2)
        fig.suptitle("Robust PCA EDA/HR Noise Results")
        axes[0].set_title('EDA Noise')
        axes[1].set_title('HR Noise')
        sns.heatmap(noise_E,ax = axes[0])
        sns.heatmap(noise_H,ax = axes[1])
    
    return score, rand , x_rank


In [38]:
train_split = .7
ms_df = pd.read_csv('MS_DateTime_Complete_test.csv')
num_subjects = HR_Distance.shape[0]
num_train = int(num_subjects * train_split)
num_test = num_subjects - num_train
# length already accounted for in normalization

# Full Length Distances


In [64]:
k_sweep = np.array([3, 15, 25, 35, 45, 60]) #6
rho_sweep = np.array([.1, 1, 10, 100]) # 4
alpha_sweep = np.array([.1, 1, 5]) # 3
beta_sweep = np.array([.1, 1, 5]) #3
#k_sweep = np.array([40])
#rho_sweep = np.array([1])
score_max = 0
rand_max = 0

HR_Full_Laplace = np.empty(HR_Distance.shape)
EDA_Full_Laplace = np.empty(EDA_Distance.shape)
#HR_Windows_Laplace = np.empty((15,73,73))
#EDA_Windows_Laplace = np.empty((15,73,73))

k_max = np.array([3,3])
rho_max = np.array([.1,.1])
alpha_max = np.array([.1, .1])
beta_max = np.array([.1, .1])
rank_avg = 0
score_avg = 0
rand_avg = 0
           
                
for alpha in alpha_sweep:
    for beta in beta_sweep:
        print(alpha, beta)
        
        #for i in range(15):
        #    HR_Full_Laplace[i,:,:] = distance_laplace(HR_Distance[i,:,:], scale_up=a, smooth_out=beta)
        #    EDA_Full_Laplace[i,:,:] = distance_laplace(EDA_Distance[i,:,:], scale_up=a, smooth_out=beta)
            
        HR_Full_Laplace = distance_laplace(HR_Distance, scale_up=a, smooth_out=beta)
        EDA_Full_Laplace = distance_laplace(EDA_Distance, scale_up=a, smooth_out=beta)
        ms_labels = ms_interval_labels[i,:]
        for k in k_sweep:
            for Rho in rho_sweep:
               

                    #Rho= 1
                    #k = 20
                   
                    
                out_score, out_rand, out_rank = triplePCA(Rho, k, HR_Full_Laplace, EDA_Full_Laplace,ms_end_labels,ld = 1/73,printfigs=False)
                score_avg = score_avg + out_score
                rand_avg = rand_avg + out_rand
                rank_avg = rank_avg + out_rank

                
   

                if out_score > score_max:
                    score_max = out_score
                    k_max[0] = k
                    rho_max[0] = Rho
                    alpha_max[0] = alpha
                    beta_max[0] = beta
                if out_rand > rand_max:
                    rand_max = out_rand
                    k_max[1] = k
                    rho_max[1] = Rho
                    alpha_max[1] = alpha
                    beta_max[1] = beta
                    

score_avg = score_avg / (6*4*3*3)         
rand_avg = rand_avg / (6*4*3*3) 
rank_avg = rank_avg / (6*4*3*3)  

print("Accuracy Max: " + str(score_max))
print("Rand Max: " + str(rand_max))
print("K Maxes: " + str(k_max[0]) + ", " + str(k_max[1]))
print("Rho Maxes: " + str(rho_max[0]) + ", " + str(rho_max[1]))
print("Alpha Maxes: " + str(alpha_max[0]) + ", " + str(alpha_max[1]))
print("Beta Maxes: " + str(beta_max[0]) + ", " + str(beta_max[1]))
print("Average Rank: " + str(rank_avg))
print("Average Accuracy: " + str(score_avg))
print("Average Rand: " + str(rand_avg))

0.1 0.1
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 47
k set too high for out matrix of rank: 58
0.1 1.0
0.1 5.0
1.0 0.1
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 47
k set too high for out matrix of rank: 58
1.0 1.0
1.0 5.0
5.0 0.1
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 22
k set too high for out matrix of rank: 47
k set too high for out matrix of rank: 58
5.0 1.0
5.0 5.0
Accuracy Max: 0.45454545454545453
Rand Max: 0.004517967609647598
K Maxes: 3, 25
Rho Maxes: 10.0, 0.1
Alpha Maxes: 0.1, 0.1
Beta Maxes: 0.1, 5.0
Average Rank: 59.66666666666

# Windowed Distance PCA

In [71]:
k_sweep = np.array([3, 15, 25, 35, 45, 60]) #6
rho_sweep = np.array([.1, 1, 10, 100]) # 4
alpha_sweep = np.array([.1, 1, 5]) # 3
beta_sweep = np.array([.1, 1, 5]) #3
#k_sweep = np.array([40])
#rho_sweep = np.array([1])
score_max = 0
rand_max = -1

#HR_Full_Laplace = np.empty(HR_Distance.shape)
#EDA_Full_Laplace = np.empty(EDA_Distance.shape)
HR_Windows_Laplace = np.empty((15,73,73))
EDA_Windows_Laplace = np.empty((15,73,73))

k_max = np.array([3,3])
rho_max = np.array([.1,.1])
alpha_max = np.array([.1, .1])
beta_max = np.array([.1, .1])
rank_avg = 0
score_avg = 0
rand_avg = 0
rank_best =  np.array([0, 0])
for alpha in alpha_sweep:
    for beta in beta_sweep:
        print(alpha, beta)
        
        for j in range(15):
            HR_Windows_Laplace[j,:,:] = distance_laplace(HR_Distance_Windows[j,:,:], scale_up=a, smooth_out=beta)
            EDA_Windows_Laplace[j,:,:] = distance_laplace(EDA_Distance_Windows[j,:,:], scale_up=a, smooth_out=beta)
            
       
        for k in k_sweep:
            for Rho in rho_sweep:
                score_avg = 0
                rand_avg = 0
                rank_avg = 0

                    
                    
                    
                for i in range(15):
                    
                    HR_Data = HR_Windows_Laplace[i,:,:]
                    EDA_Data = EDA_Windows_Laplace[i,:,:]
                    ms_labels = ms_interval_labels[i,:]
                    
                    out_score, out_rand, out_rank = triplePCA(Rho, k, HR_Data, EDA_Data,ms_labels,ld =1/73,printfigs=False)
                    score_avg = score_avg + out_score
                    rand_avg = rand_avg + out_rand
                    rank_avg = rank_avg + out_rank


   
                score_avg = score_avg / 15
                rand_avg = rand_avg / 15
                rank_avg = rank_avg / 15
        
                if score_avg > score_max:
                    score_max = score_avg
                    k_max[0] = k
                    rho_max[0] = Rho
                    alpha_max[0] = alpha
                    beta_max[0] = beta
                    rank_best[0] = rank_avg
                if rand_avg > rand_max:
                    rand_max = rand_avg
                    k_max[1] = k
                    rho_max[1] = Rho
                    alpha_max[1] = alpha
                    beta_max[1] = beta
                    rank_best[1] = rank_avg
                    

#score_avg = score_avg / (6*4*3*3)         
#rand_avg = rand_avg / (6*4*3*3) 
#rank_avg = rank_avg / (6*4*3*3)  

print("Accuracy Max: " + str(score_max))
print("Rand Max: " + str(rand_max))
print("K Maxes: " + str(k_max[0]) + ", " + str(k_max[1]))
print("Rho Maxes: " + str(rho_max[0]) + ", " + str(rho_max[1]))
print("Alpha Maxes: " + str(alpha_max[0]) + ", " + str(alpha_max[1]))
print("Beta Maxes: " + str(beta_max[0]) + ", " + str(beta_max[1]))
print("Best Rank: " + str(rank_best[0])+ ", " + str(rank_best[1]))


0.1 0.1
k set too high for out matrix of rank: 47
k set too high for out matrix of rank: 58
0.1 1.0
0.1 5.0
1.0 0.1
k set too high for out matrix of rank: 47
k set too high for out matrix of rank: 58
1.0 1.0
1.0 5.0
5.0 0.1
k set too high for out matrix of rank: 47
k set too high for out matrix of rank: 58
5.0 1.0
5.0 5.0
Accuracy Max: 0.609090909090909
Rand Max: 0.0772865429776989
K Maxes: 3, 3
Rho Maxes: 0.1, 0.1
Alpha Maxes: 0.1, 0.1
Beta Maxes: 1.0, 1.0
Best Rank: 67, 67
